In [1]:
import numpy as np
import os, sys
from pathlib import Path
import uproot
import mplhep as mh
import matplotlib.pyplot as plt
from hist import Hist
from hist.axis import Regular

sys.path.append("/users/hep/eigen1907/Workspace/Workspace-RPC/modules")
from NanoAODTnP.Plotting.PlotEfficiency import init_figure, hist_eff_by_roll, get_region_params

In [2]:
def hist_tnp_mass(input_path1, input_path2, output_path):
    mass_2022 = uproot.open(f"{input_path1}:muon_tree/dimuon_mass").array(library="np")
    mass_2023 = uproot.open(f"{input_path2}:muon_tree/dimuon_mass").array(library="np")

    mh.style.use(mh.styles.CMS)
    fig, ax = plt.subplots(figsize=(12, 8))
    mh.cms.label(ax=ax, llabel=f'Work in Progress', com=r'$\sqrt{s} = 13.6$', year="2022, 2023", loc=2, fontsize=22)

    ax.set_xlabel('Tag-Probe Mass [GeV]', fontsize=22)
    ax.set_ylabel('Events/0.5GeV', fontsize=22)
    ax.set_xlim(65, 115)

    h_mass_2022 = Hist(Regular(80, 70, 110))
    h_mass_2023 = Hist(Regular(80, 70, 110))

    h_mass_2022.fill(mass_2022)
    h_mass_2023.fill(mass_2023)

    h_mass_2022.plot(ax = ax,
                     yerr = False,
                     histtype="fill",
                     edgecolor='black',
                     facecolor='gray',
                     linewidth=1.5,
                     hatch='\\\\',
                     flow=None,
                     label=f"2022",
                     alpha=0.6)

    h_mass_2023.plot(ax = ax,
                     yerr = False,
                     histtype="fill",
                     edgecolor='black',
                     facecolor='black',
                     linewidth=1.5,
                     hatch='///',
                     flow=None,
                     label=f"2023",
                     alpha=0.6)

    ax.legend(fontsize=24)

    if not output_path.parent.exists():
        output_path.parent.mkdir(parents=True)
    fig.savefig(output_path)
    plt.close(fig)

def hist_tnp_mass_pass_fail(input_path1, input_path2, output_path):
    mass_2022 = uproot.open(f"{input_path1}:tree/dimuon_mass").array(library="np")
    mass_2023 = uproot.open(f"{input_path2}:tree/dimuon_mass").array(library="np")
    
    is_matched_2022 = uproot.open(f"{input_path1}:tree/is_matched").array(library="np")
    is_matched_2023 = uproot.open(f"{input_path2}:tree/is_matched").array(library="np")

    pass_mass_2022 = mass_2022[is_matched_2022]
    pass_mass_2023 = mass_2023[is_matched_2023]

    fail_mass_2022 = mass_2022[~is_matched_2022]
    fail_mass_2023 = mass_2023[~is_matched_2023]

    mass_run3 = np.concatenate((mass_2022, mass_2023))
    matched_mass_run3 = np.concatenate((pass_mass_2022, pass_mass_2023))
    unmatched_mass_run3 = np.concatenate((fail_mass_2022, fail_mass_2023))

    mh.style.use(mh.styles.CMS)
    fig, ax = plt.subplots(figsize=(12, 8))
    mh.cms.label(ax=ax, llabel=f'Work in Progress', com=r'$\sqrt{s} = 13.6$', year="2022, 2023", loc=2, fontsize=22)

    ax.set_xlabel('Tag-Probe Mass [GeV]', fontsize=22)
    ax.set_ylabel('Events/0.5GeV', fontsize=22)
    ax.set_xlim(65, 115)

    h_mass_run3 = Hist(Regular(80, 70, 110))
    h_matched_mass_run3 = Hist(Regular(80, 70, 110))
    h_unmatched_mass_run3 = Hist(Regular(80, 70, 110))

    h_mass_run3.fill(mass_run3)
    h_matched_mass_run3.fill(matched_mass_run3)
    h_unmatched_mass_run3.fill(unmatched_mass_run3)

    h_matched_mass_run3.plot(ax = ax,
                             yerr = False,
                             histtype="fill",
                             edgecolor='black',
                             facecolor='gray',
                             linewidth=1.5,
                             hatch='\\\\',
                             flow=None,
                             label=r"$m_{\mu\mu}^{pass}$",
                             alpha=0.6)

    h_unmatched_mass_run3.plot(ax = ax,
                               yerr = False,
                               histtype="fill",
                               edgecolor='black',
                               facecolor='black',
                               linewidth=1.5,
                               hatch='///',
                               flow=None,
                               label=r"$m_{\mu\mu}^{fail}$",
                               alpha=0.6)

    ax.legend(fontsize=24)

    if not output_path.parent.exists():
        output_path.parent.mkdir(parents=True)
    fig.savefig(output_path)
    plt.close(fig)

In [3]:
store_path = Path('/users/hep/eigen1907/store/TnP-NanoAOD/analysis')
output_path = Path('/users/hep/eigen1907/Workspace/Workspace-RPC/240425-TnP_RPC24/plotting/hist_mass_by_event')

data_type_list = ['without_blacklist', 'with_blacklist_roll', 'with_blacklist_roll_run']
#data_type_list = ['without_blacklist']

In [4]:
for data_type in data_type_list:
    input_path_2022 = store_path / data_type / 'Run2022.root'
    input_path_2023 = store_path / data_type / 'Run2023.root'
    hist_tnp_mass(input_path_2022, input_path_2023, output_path / data_type / 'run3_tnp_mass_2022_2023.png')
    #hist_tnp_mass_pass_fail(input_path_2022, input_path_2023, output_path / data_type / 'run3_tnp_mass_pass_fail.png')